In [3]:
filename = './data1/kakaoAPI.txt'
with open(filename) as f:
    api_key = f.read()

In [4]:
len(api_key)

32

In [5]:
import requests
from urllib.parse import quote

In [7]:
addr = '서울시 종로구 종로1길 36(수송동)'
temp = quote(addr)

In [11]:
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
url = f'{search_url}?query={temp}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%EC%8B%9C%20%EC%A2%85%EB%A1%9C%EA%B5%AC%20%EC%A2%85%EB%A1%9C1%EA%B8%B8%2036%28%EC%88%98%EC%86%A1%EB%8F%99%29'

In [34]:
result = requests.get(url, headers={"Authorization" : f'KakaoAK {api_key}'}).json()
#result = requests.get(url, headers={"Authorization" : f'KakaoAK {api_key}'}).json()

In [36]:
#위코드에서 json 삭제후 확인
#result.status_code

In [37]:
result

{'documents': [{'address': {'address_name': '서울 종로구 수송동 146-12',
    'b_code': '1111012400',
    'h_code': '1111061500',
    'main_address_no': '146',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_h_name': '종로1.2.3.4가동',
    'region_3depth_name': '수송동',
    'sub_address_no': '12',
    'x': '126.978988255925',
    'y': '37.5735051436739'},
   'address_name': '서울 종로구 종로1길 36',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 종로구 종로1길 36',
    'building_name': '종로구청',
    'main_building_no': '36',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_name': '수송동',
    'road_name': '종로1길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.978988255925',
    'y': '37.5735051436739',
    'zone_no': '03152'},
   'x': '126.978988255925',
   'y': '37.5735051436739'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [42]:
# .json으로 안가져온경우 밑 코드로 다시 부럴오기
# import json
# res = json.load(result.text)

In [43]:
result.keys()

dict_keys(['documents', 'meta'])

In [54]:
print(result['documents'][0]['x'])
print(result['documents'][0]['y'])

126.978988255925
37.5735051436739


In [56]:
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])

In [61]:
print(lng, lat)

126.978988255925 37.5735051436739


In [62]:
import pandas as pd
df = pd.read_csv('./data1/공공기관.csv')
df

,공공기관,도로명주소
0,종로구청,서울특별시 종로구 종로1길 36(수송동)
1,노원구청,서울특별시 노원구 노해로 437(상계동)
2,송파구청,서울특별시 송파구 올림픽로 326(신천동)
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동)
4,양천구청,서울특별시 양천구 목동동로 105(신정동)


In [66]:
lat_list = [] 
lng_list = []
for i in df.index:
    search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    url = f'{search_url}?query={df.도로명주소[i]}'
    result = requests.get(url, headers={"Authorization" : f'KakaoAK {api_key}'}).json()
    lng = float(result['documents'][0]['x'])
    lat = float(result['documents'][0]['y'])
    lat_list.append(lat)
    lng_list.append(lng)

In [76]:
df['위도'] = lat_list
df['경도'] = lng_list
df.to_csv('공공기관2.csv', index=False , encoding='cp949')

In [68]:
df

,공공기관,도로명주소,위도,경도
0,종로구청,서울특별시 종로구 종로1길 36(수송동),37.573505,126.978988
1,노원구청,서울특별시 노원구 노해로 437(상계동),37.654523,127.056268
2,송파구청,서울특별시 송파구 올림픽로 326(신천동),37.514477,127.105860
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동),37.563426,126.903357
4,양천구청,서울특별시 양천구 목동동로 105(신정동),37.517075,126.866543


In [69]:
import folium

In [77]:
map = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=12)
for i in df.index:
    folium.Circle( radius=100, location=[df.위도[i], df.경도[i]], popup=folium.Popup(df.도로명주소[i], max_width=200),
                  tooltip=df.공공기관[i], color = 'red', fill=True, fill_color = '#3186cc').add_to(map)
title = '<h3 align="center" style="font-size:20"> 서울시 구청</h3>'
map.get_root().html.add_child(folium.Element(title))
map